In [9]:
##### ----------------- ESTE PRIMER BLOQUE IGNORENLO YA QUE ERA PARA PODER DESACTIVAR LAS CONEXIONES, BORRAR LA BASE Y CREAR UNA NUEVA ------------------

import psycopg2

# Datos de conexión
DB_HOST = "localhost"
DB_USER = "postgres"
DB_PASSWORD = "Spriest123"
DB_NAME = "fleetlogix"

# Conectarse a otra base, por ejemplo 'postgres'
conn = psycopg2.connect(
    dbname="postgres",      
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST
)
conn.autocommit = True
cur = conn.cursor()

# Terminar todas las conexiones activas a la base 'nueva'
cur.execute(f"""
SELECT pg_terminate_backend(pid)
FROM pg_stat_activity
WHERE datname = '{DB_NAME}';
""")
print(f"Conexiones a {DB_NAME} terminadas.")

# Eliminar la base si existe
cur.execute(f"DROP DATABASE IF EXISTS {DB_NAME};")
print(f"Base {DB_NAME} eliminada.")

# Crear la base de nuevo
cur.execute(f"CREATE DATABASE {DB_NAME};")
print(f"Base {DB_NAME} creada.")

# Cerrar conexión
cur.close()
conn.close()

Conexiones a fleetlogix terminadas.
Base fleetlogix eliminada.
Base fleetlogix creada.


In [9]:
import psycopg2
from faker import Faker
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ==============================
# ⚙️ Configuración inicial
# ==============================
fake = Faker("es_ES")
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
BATCH = 5000

# ==============================
# 🔌 Conexión PostgreSQL
# ==============================
conn = psycopg2.connect(
    dbname="fleetlogix",
    user="postgres",
    password="Spriest123",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# ==============================
# 🧾 Función de logs
# ==============================
def log_load(table_name, records_inserted, status, error_message=None):
    cur_log = conn.cursor()
    cur_log.execute("""
        INSERT INTO load_logs (table_name, records_inserted, end_time, status, error_message)
        VALUES (%s,%s,%s,%s,%s)
    """, (table_name, records_inserted, datetime.now(), status, error_message))
    conn.commit()
    cur_log.close()

# ==============================
# 🧹 Funciones auxiliares
# ==============================
def clean_value(v):
    if isinstance(v, (np.generic, np.bool_)):
        return v.item()
    if isinstance(v, (np.datetime64, pd.Timestamp)):
        return pd.to_datetime(v).to_pydatetime()
    if pd.isna(v):
        return None
    return v

def clean_row(row):
    return tuple(clean_value(x) for x in row)

# ==============================
# 🚗 1️⃣ VEHICLES
# ==============================
vehicle_types = ["Camión grande", "Camión mediano", "Van", "Motocicleta"]
fuel_types = ["Diesel", "Nafta", "Eléctrico"]
vehicle_capacity_ranges = {
    "Camión grande": (12000, 20000),
    "Camión mediano": (6000, 12000),
    "Van": (1000, 4000),
    "Motocicleta": (50, 200)
}

def sample_capacity_for_type(vtype):
    low, high = vehicle_capacity_ranges[vtype]
    return random.randint(low, high)

vehicles_data = []
for _ in range(200):
    vt = random.choice(vehicle_types)
    status = random.choice(["active", "maintenance", "retired"])
    vehicles_data.append((
        fake.unique.license_plate(),
        vt,
        sample_capacity_for_type(vt),
        random.choice(fuel_types),
        fake.date_between(start_date="-10y", end_date="-5y"),
        status
    ))

for i in range(0, len(vehicles_data), BATCH):
    chunk = [clean_row(row) for row in vehicles_data[i:i+BATCH]]
    cur.executemany("""
        INSERT INTO vehicles (license_plate, vehicle_type, capacity_kg, fuel_type, acquisition_date, status)
        VALUES (%s,%s,%s,%s,%s,%s)
    """, chunk)
    conn.commit()
log_load("vehicles", len(vehicles_data), "success")
print("✅ Vehicles insertados")

vehicle_id_to_type = {i+1: vehicles_data[i][1] for i in range(len(vehicles_data))}
vehicle_id_to_status = {i+1: vehicles_data[i][5] for i in range(len(vehicles_data))}

# ==============================
# 🧍‍♂️ 2️⃣ DRIVERS
# ==============================
drivers_data = []
for i in range(400):
    hire_date = fake.date_between(start_date="-6y", end_date="-5y")
    license_expiry = fake.date_between(start_date="today", end_date="+1y")
    drivers_data.append((
        f"EMP{i+1000}",
        fake.first_name(),
        fake.last_name(),
        fake.unique.bothify(text="LIC#####"),
        license_expiry,
        fake.phone_number(),
        hire_date,
        random.choice(["active","inactive","suspended"])
    ))

for i in range(0, len(drivers_data), BATCH):
    chunk = [clean_row(row) for row in drivers_data[i:i+BATCH]]
    cur.executemany("""
        INSERT INTO drivers (employee_code, first_name, last_name, license_number, license_expiry, phone, hire_date, status)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """, chunk)
    conn.commit()
log_load("drivers", len(drivers_data), "success")
print("✅ Drivers insertados")

# ==============================
# 🗺️ 3️⃣ ROUTES
# ==============================
cities = ["Buenos Aires", "Rosario", "Córdoba", "Mendoza", "La Plata", "Salta", "Mar del Plata", "Bahía Blanca"]
routes_data = []
for i in range(50):
    origin, destination = random.sample(cities, 2)
    routes_data.append((
        f"R{i+1:04d}",
        origin,
        destination,
        round(random.uniform(20,1200),2),
        round(max(0.5, random.uniform(20,1200)/random.uniform(40,90)),2),
        round(random.uniform(0,5000),2)
    ))

for i in range(0, len(routes_data), BATCH):
    chunk = [clean_row(row) for row in routes_data[i:i+BATCH]]
    cur.executemany("""
        INSERT INTO routes (route_code, origin_city, destination_city, distance_km, estimated_duration_hours, toll_cost)
        VALUES (%s,%s,%s,%s,%s,%s)
    """, chunk)
    conn.commit()
log_load("routes", len(routes_data), "success")
print("✅ Routes insertadas")

routes_df = pd.DataFrame(routes_data, columns=[
    "route_code", "origin_city", "destination_city", "distance_km", "estimated_duration_hours", "toll_cost"
])
routes_df["route_id"] = routes_df.index + 1

# ==============================
# ⚙️ Funciones auxiliares trips
# ==============================
def get_hourly_distribution():
    hours = list(range(24))
    weights = [0.02, 0.03, 0.05, 0.06, 0.08, 0.10, 0.12, 0.10, 0.09, 0.06, 0.05, 0.04,
               0.03, 0.02, 0.02, 0.02, 0.03, 0.04, 0.06, 0.07, 0.08, 0.05, 0.03, 0.02]
    return int(np.random.choice(hours, p=np.array(weights)/sum(weights)))

def generate_trips(n_trips, start_date, end_date, routes_df, vehicle_id_to_type, vehicle_id_to_status):
    fuel_consumption_ranges = {
        "Camión grande": (0.25, 0.35),
        "Camión mediano": (0.18, 0.28),
        "Van": (0.12, 0.20),
        "Motocicleta": (0.05, 0.10)
    }

    def sample_fuel_rate(vtype):
        low, high = fuel_consumption_ranges[vtype]
        return random.uniform(low, high)

    total_days = (end_date - start_date).days
    trip_data = []

    for _ in range(n_trips):
        vid = random.randint(1, len(vehicle_id_to_type))
        did = random.randint(1, len(drivers_data))
        rid = random.randint(1, len(routes_df))

        dep = start_date + timedelta(
            days=random.randint(0, total_days),
            hours=get_hourly_distribution(),
            minutes=random.randint(0, 59)
        )

        route = routes_df.iloc[rid-1]
        duration = route["estimated_duration_hours"]
        arr = dep + timedelta(hours=float(duration) + random.uniform(-0.2, 1.0))

        status = np.random.choice(['completed','in_progress','cancelled'], p=[0.85,0.1,0.05])

        if status == 'cancelled' or vehicle_id_to_status[vid] == "maintenance":
            arr = None
            fuel = 0
            if vehicle_id_to_status[vid] == "maintenance":
                status = "cancelled"
        else:
            fuel = round(route["distance_km"] * sample_fuel_rate(vehicle_id_to_type[vid]), 2)

        trip_data.append((vid, did, rid, dep, arr, fuel, status))

    return trip_data

# ==============================
# 🚚 4️⃣ TRIPS
# ==============================
trip_data = generate_trips(
    n_trips=50000,
    start_date=pd.to_datetime('2023-01-01'),
    end_date=pd.to_datetime(datetime.today().strftime('%Y-%m-%d')),
    routes_df=routes_df,
    vehicle_id_to_type=vehicle_id_to_type,
    vehicle_id_to_status=vehicle_id_to_status
)

for i in range(0, len(trip_data), BATCH):
    chunk = [clean_row(row) for row in trip_data[i:i+BATCH]]
    cur.executemany("""
        INSERT INTO trips (vehicle_id, driver_id, route_id, departure_datetime, arrival_datetime, fuel_consumed_liters, status)
        VALUES (%s,%s,%s,%s,%s,%s,%s)
    """, chunk)
    conn.commit()
log_load("trips", len(trip_data), "success")
print("✅ Trips insertados")

# ==============================
# 📦 5️⃣ DELIVERIES
# ==============================
deliveries_data = []
for _ in range(1000):
    trip_id = random.randint(1, len(trip_data))
    tracking_number = fake.unique.bothify(text="TRK#####")
    customer_name = fake.company()
    delivery_address = fake.address().replace("\n", ", ")
    package_weight_kg = round(random.uniform(10, 500), 2)
    revenue = round(package_weight_kg * random.uniform(5, 20), 2)
    status = random.choice(["pending", "delivered", "failed"])
    delivered_datetime = None
    if status == "delivered":
        delivered_datetime = datetime.now() - timedelta(days=random.randint(1, 60))

    deliveries_data.append((
        trip_id,
        tracking_number,
        customer_name,
        delivery_address,
        package_weight_kg,
        revenue,
        delivered_datetime,
        status
    ))

for i in range(0, len(deliveries_data), BATCH):
    chunk = [clean_row(row) for row in deliveries_data[i:i+BATCH]]
    cur.executemany("""
        INSERT INTO deliveries (
            trip_id,
            tracking_number,
            customer_name,
            delivery_address,
            package_weight_kg,
            revenue,
            delivered_datetime,
            status
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """, chunk)
    conn.commit()
log_load("deliveries", len(deliveries_data), "success")
print("✅ Deliveries insertados")

# ==============================
# 🧰 6️⃣ MAINTENANCE
# ==============================
maintenance_types = ["Cambio de aceite","Revisión de frenos","Cambio de llantas","Revisión general","Motor","Alineación"]
maintenance_data = []

for vid in range(1, len(vehicle_id_to_type)+1):
    n_maint = random.randint(1, 5)
    for _ in range(n_maint):
        maint_type = random.choice(maintenance_types)
        maint_date = fake.date_between(start_date="-2y", end_date="today")
        cost = round(random.uniform(100, 2000),2)
        maintenance_data.append((vid, maint_type, maint_date, cost))

for i in range(0, len(maintenance_data), BATCH):
    chunk = [clean_row(row) for row in maintenance_data[i:i+BATCH]]
    cur.executemany("""
        INSERT INTO maintenance (vehicle_id, maintenance_type, maintenance_date, cost)
        VALUES (%s,%s,%s,%s)
    """, chunk)
    conn.commit()
log_load("maintenance", len(maintenance_data), "success")
print("✅ Maintenance insertado")

# ==============================
# Cerrar conexión
# ==============================
cur.close()
conn.close()
print("✅ Script completo finalizado")


✅ Vehicles insertados
✅ Drivers insertados
✅ Routes insertadas
✅ Trips insertados
✅ Deliveries insertados
✅ Maintenance insertado
✅ Script completo finalizado
